In [1]:
print("all ok")

all ok


### AutoGen Teams

In [2]:
import asyncio

from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

model_client=OpenAIChatCompletionClient(model='gpt-4o')

In [4]:
from autogen_agentchat.agents import AssistantAgent

dsa_solver=AssistantAgent(
    name="Complex_DSA_Solver",
    model_client=model_client,
    description="A DSA Solver agent.",
    system_message="You give code in python to solve complex DSA problems. Give under 100 words."
)

code_reviewer=AssistantAgent(
    name="Code_Reviewer_system",
    model_client=model_client,
    description="A code reviewer agent.",
    system_message="You review the code given by the Complex_DSA_Solver and make sure it is optimized. Give under 10 words"
)

code_editor=AssistantAgent(
    name="Code_Editor_system",
    model_client=model_client,
    description="A code editor agent.",
    system_message="You make the code easy to understand and add comments wherever required. Give under 10 words"
)

In [ ]:
from autogen_agentchat.teams import RoundRobinGroupChat

teams=RoundRobinGroupChat(
    participants=[dsa_solver,code_reviewer,code_editor],    # maintain the order of agents
    max_turns=3  # no of execution of the teams agent except user message 
)

In [6]:
from autogen_agentchat.messages import TextMessage

In [7]:
async def run_team():
    task=TextMessage(content="Write a simple python code to add 2 numbers",source='user')
    result=await teams.run(task=task)
    print(result)

In [8]:
if (__name__)=="__main__":
    await run_team()

messages=[TextMessage(id='a1451f46-a356-4177-ab2d-8c0007556fc3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 23, 5, 19, 35, 133791, tzinfo=datetime.timezone.utc), content='Write a simple python code to add 2 numbers', type='TextMessage'), TextMessage(id='ea7cd551-7a48-4b6c-8f27-d0c953828d7b', source='Complex_DSA_Solver', models_usage=RequestUsage(prompt_tokens=40, completion_tokens=75), metadata={}, created_at=datetime.datetime(2025, 7, 23, 5, 19, 38, 148848, tzinfo=datetime.timezone.utc), content='```python\n# Function to add two numbers\ndef add_numbers(num1, num2):\n    return num1 + num2\n\n# Example usage\nnumber1 = 5\nnumber2 = 10\nresult = add_numbers(number1, number2)\nprint("The sum is:", result)\n```\nThis function takes two numbers as input and returns their sum.', type='TextMessage'), TextMessage(id='9bba2a4e-6f7d-4525-8b98-335a6ea884ba', source='Code_Reviewer_system', models_usage=RequestUsage(prompt_tokens=131, completion_tokens=6)

messages=[
    
    TextMessage(id='a1451f46-a356-4177-ab2d-8c0007556fc3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 23, 5, 19, 35, 133791, tzinfo=datetime.timezone.utc), content='Write a simple python code to add 2 numbers', type='TextMessage'), 
    
    TextMessage(id='ea7cd551-7a48-4b6c-8f27-d0c953828d7b', source='Complex_DSA_Solver', models_usage=RequestUsage(prompt_tokens=40, completion_tokens=75), metadata={}, created_at=datetime.datetime(2025, 7, 23, 5, 19, 38, 148848, tzinfo=datetime.timezone.utc), content='```python\n# Function to add two numbers\ndef add_numbers(num1, num2):\n    return num1 + num2\n\n# Example usage\nnumber1 = 5\nnumber2 = 10\nresult = add_numbers(number1, number2)\nprint("The sum is:", result)\n```\nThis function takes two numbers as input and returns their sum.', type='TextMessage'), 
    
    TextMessage(id='9bba2a4e-6f7d-4525-8b98-335a6ea884ba', source='Code_Reviewer_system', models_usage=RequestUsage(prompt_tokens=131, completion_tokens=6), metadata={}, created_at=datetime.datetime(2025, 7, 23, 5, 19, 39, 219726, tzinfo=datetime.timezone.utc), content='Code is optimized and concise.', type='TextMessage'), 
    
    TextMessage(id='94035049-2cc7-4e0f-8fa9-c56d478ad3d3', source='Code_Editor_system', models_usage=RequestUsage(prompt_tokens=140, completion_tokens=9), metadata={}, created_at=datetime.datetime(2025, 7, 23, 5, 19, 40, 80528, tzinfo=datetime.timezone.utc), content='Thank you! Glad you found it efficient.', type='TextMessage')] 
    
    stop_reason='Maximum number of turns 3 reached.'

In [9]:
teams=RoundRobinGroupChat(
    participants=[dsa_solver,code_reviewer,code_editor],    # maintain the order of agents
    max_turns=9  # no of execution of the teams agent except user message 
)

async def run_team():
    task=TextMessage(content="Write a simple python code to add 2 numbers",source='user')
    result=await teams.run(task=task)
    for each_message_agent in result.messages:
        print(f"{each_message_agent.source}")

if (__name__)=="__main__":
    await run_team()

user
Complex_DSA_Solver
Code_Reviewer_system
Code_Editor_system
Complex_DSA_Solver
Code_Reviewer_system
Code_Editor_system
Complex_DSA_Solver
Code_Reviewer_system
Code_Editor_system


## Termination

In [10]:
dsa_solver=AssistantAgent(
    name="Complex_DSA_Solver",
    model_client=model_client,
    description="A DSA Solver agent.",
    system_message="You give code in python to solve complex DSA problems. Give under 100 words."
)

code_reviewer=AssistantAgent(
    name="Code_Reviewer_system",
    model_client=model_client,
    description="A code reviewer agent.",
    system_message="You review the code given by the Complex_DSA_Solver and make sure it is optimized. Give under 10 words." \
    "If you feel that the code is fine then please say 'TERMINATE'"
)

code_editor=AssistantAgent(
    name="Code_Editor_system",
    model_client=model_client,
    description="A code editor agent.",
    system_message="You make the code easy to understand and add comments wherever required. Give under 10 words"
)

In [ ]:
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination

my_termination=TextMentionTermination(text='TERMINATE') | MaxMessageTermination(max_messages=3)  # MaxMessageTermination include user message also

In [16]:
teams=RoundRobinGroupChat(
    participants=[dsa_solver,code_reviewer,code_editor],    # maintain the order of agents
    termination_condition=my_termination,
    max_turns=9  # no of execution of the teams agent except user message 
)

async def run_team():
    task=TextMessage(content="Write a simple python code to add 2 numbers",source='user')
    result=await teams.run(task=task)
    for each_message_agent in result.messages:
        print(f"{each_message_agent.source} : {each_message_agent.content}")

if (__name__)=="__main__":
    await run_team()

user : Write a simple python code to add 2 numbers
Complex_DSA_Solver : ```python
# Input two numbers
num1 = float(input("Enter first number: "))
num2 = float(input("Enter second number: "))

# Calculate the sum
sum_result = num1 + num2

# Display the sum
print("The sum is:", sum_result)
```

This code takes two numbers as input from the user, calculates their sum, and prints the result.
Code_Reviewer_system : TERMINATE


### Running as stream

In [19]:
from autogen_agentchat.base import TaskResult

async for message in teams.run_stream(task="Write a python code for fibonacci series."):
    if isinstance(message,TaskResult):
        print("Stop reason:",message.stop_reason)
    else:
        print(f"{message.source} : {message.content}")

user : Write a python code for fibonacci series.
Code_Editor_system : ```python
# Function to generate Fibonacci series
def fibonacci_series(n):
    # Initialize the first two numbers in the series
    a, b = 0, 1
    # Iterate up to n
    for _ in range(n):
        # Print the current number
        print(a, end=" ")
        # Update numbers for the next in series
        a, b = b, a + b

# Specify number of terms
num_terms = int(input("Enter number of terms: "))
# Generate Fibonacci series
fibonacci_series(num_terms)
```

Generates Fibonacci up to specified terms.
Complex_DSA_Solver : This Python code generates the Fibonacci series for a specified number of terms. It uses a loop to print each term by updating two variables, `a` and `b`, where `a` holds the current term and `b` holds the next.
Stop reason: Maximum number of messages 3 reached, current message count: 3


### Resuming the team

In [20]:
from autogen_agentchat.agents import AssistantAgent

add_1_agent_first=AssistantAgent(
    name="add_1_agent_first",
    model_client=model_client,
    description="add 1 with the given number",
    system_message="Add 1 to the number, first number is 0. Give the result output"
)

add_1_agent_second=AssistantAgent(
    name="add_1_agent_second",
    model_client=model_client,
    description="add 1 with the given number",
    system_message="Add 1 to the number you got from the previous run. Give the result output"
)


add_1_agent_third=AssistantAgent(
    name="add_1_agent_third",
    model_client=model_client,
    description="add 1 with the given number",
    system_message="Add 1 to the number you got from the previous run. Give the result output"
)

my_increament_team=RoundRobinGroupChat(participants=[add_1_agent_first,add_1_agent_second,add_1_agent_third],max_turns=3)

In [21]:
from autogen_agentchat.ui import Console

await Console(my_increament_team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
2
---------- TextMessage (add_1_agent_third) ----------
3


TaskResult(messages=[TextMessage(id='5e44c079-d112-45fe-9ea2-8caffbd50ba3', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 23, 7, 1, 53, 887414, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='1a8d12d8-30e6-4e7f-8e28-ec5b1a143941', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=34, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 23, 7, 1, 54, 554122, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(id='e2234500-d28b-444e-9e5a-923d971d556b', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=44, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 23, 7, 1, 55, 615366, tzinfo=datetime.timezone.utc), content='3', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [22]:
await Console(my_increament_team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
4
---------- TextMessage (add_1_agent_second) ----------
5
---------- TextMessage (add_1_agent_third) ----------
6


TaskResult(messages=[TextMessage(id='b4d66448-a9de-4c0d-aa08-8d760b11cbe3', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=50, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 23, 7, 2, 5, 648642, tzinfo=datetime.timezone.utc), content='4', type='TextMessage'), TextMessage(id='c95a594b-65af-43e2-8c0f-66c9b492d1a3', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=60, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 23, 7, 2, 6, 510029, tzinfo=datetime.timezone.utc), content='5', type='TextMessage'), TextMessage(id='865a20f2-976a-4ba3-a95e-dda47d97a797', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=69, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 23, 7, 2, 7, 457828, tzinfo=datetime.timezone.utc), content='6', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [23]:
await my_increament_team.reset()
await Console(my_increament_team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
The first number is 0. Adding 1 to it gives the result: 1.
---------- TextMessage (add_1_agent_second) ----------
The previous result was 1. Adding 1 to it gives the result: 2.
---------- TextMessage (add_1_agent_third) ----------
The previous result was 2. Adding 1 to it gives the result: 3.


TaskResult(messages=[TextMessage(id='6d7fdad2-2423-4386-ba24-badaa04c9d37', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 7, 23, 7, 5, 8, 238987, tzinfo=datetime.timezone.utc), content='The first number is 0. Adding 1 to it gives the result: 1.', type='TextMessage'), TextMessage(id='c9230f9c-038f-4768-b3cb-c9257affa72c', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=52, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 7, 23, 7, 5, 9, 224008, tzinfo=datetime.timezone.utc), content='The previous result was 1. Adding 1 to it gives the result: 2.', type='TextMessage'), TextMessage(id='d2efaa91-9176-4a40-ad4d-45cce710fe60', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=80, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 7, 23, 7, 5, 10, 168156, tzinfo=datetime.timezone.utc), content='The previous result was

### Single Person Team

In [25]:
my_increament_team=RoundRobinGroupChat(participants=[add_1_agent_first],max_turns=5)
await my_increament_team.reset()
await Console(my_increament_team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_first) ----------
2
---------- TextMessage (add_1_agent_first) ----------
3
---------- TextMessage (add_1_agent_first) ----------
4
---------- TextMessage (add_1_agent_first) ----------
5


TaskResult(messages=[TextMessage(id='13a600d5-af40-47e3-b998-653fad96a3ee', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 23, 9, 7, 42, 990473, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='71268725-b70e-4ae8-94e9-d6a5fa139aed', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 23, 9, 7, 43, 772477, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(id='c3714d12-854e-4f04-aa5a-53fc5f4ec108', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=34, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 23, 9, 7, 45, 147614, tzinfo=datetime.timezone.utc), content='3', type='TextMessage'), TextMessage(id='5a85d976-ce94-49ee-ba55-142f93d7031f', source='add_1_agent_first', models_usage=RequestUsage(prompt_to

### Cancellation Token

In [26]:
from autogen_core import CancellationToken # manually cancel the process